In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification

class SentimentAnalysisModel:
    def __init__(self, model_name="bert-base-uncased", num_labels=2):
        self.tokenizer = BertTokenizer.from_pretrained(model_name)
        self.model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

    def preprocess(self, text):
        inputs = self.tokenizer(text, return_tensors="pt")
        return inputs

    def predict(self, text):
        inputs = self.preprocess(text)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        predicted_class = torch.argmax(logits, dim=1)
        return predicted_class.item()

    def train(self, train_dataloader, epochs=5, learning_rate=2e-5):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.train()
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=learning_rate)

        for epoch in range(epochs):
            for batch in train_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                loss, logits = outputs
                loss.backward()
                optimizer.step()
                optimizer.zero_grad()

    def evaluate(self, val_dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for batch in val_dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask, labels=labels)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

In [4]:
import torch
import random
import numpy as np
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification

class TransformerEvaluator:
    def __init__(self, model_name="distilbert-base-uncased", num_labels=2):
        self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
        self.model = DistilBertForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)
        self.original_weights = {name: param.clone() for name, param in self.model.named_parameters()}
        self.bit_flip_log = []

    def preprocess(self, text):
        return self.tokenizer(text, return_tensors="pt")

    def predict(self, text):
        inputs = self.preprocess(text)
        with torch.no_grad():
            outputs = self.model(**inputs)
        logits = outputs.logits
        return torch.argmax(logits, dim=1).item()

    def evaluate(self, dataloader):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.model.to(device)
        self.model.eval()

        correct = 0
        total = 0
        with torch.no_grad():
            for batch in dataloader:
                input_ids, attention_mask, labels = tuple(t.to(device) for t in batch)
                outputs = self.model(input_ids, attention_mask=attention_mask)
                _, predicted = torch.max(outputs.logits, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = 100 * correct / total
        return accuracy

    def inject_random_bit_flip(self):
        param_list = list(self.model.named_parameters())
        layer_index = random.randint(0, len(param_list) - 1)
        layer_name, param = param_list[layer_index]

        # Flatten the tensor and randomly select a bit
        flat_param = param.data.view(-1)
        param_index = random.randint(0, flat_param.numel() - 1)
        original_value = flat_param[param_index].item()

        # Convert to bitwise representation and flip a random bit
        param_value = flat_param[param_index].item()
        int_repr = np.float32(param_value).view(np.int32)
        bit_position = random.randint(0, 31)
        flipped_int = int_repr ^ (1 << bit_position)
        flipped_value = np.float32(flipped_int).view(np.float32)

        # Apply the flipped value
        flat_param[param_index] = flipped_value
        self.bit_flip_log.append((layer_name, param_index, bit_position, original_value, flipped_value))

        return layer_name, param_index, bit_position, original_value, flipped_value

    def reset_weights(self):
        for name, param in self.model.named_parameters():
            param.data.copy_(self.original_weights[name])
        self.bit_flip_log.clear()

    def run_experiment(self, dataloader, num_flips=10):
        results = []

        original_accuracy = self.evaluate(dataloader)
        print(f"Original accuracy: {original_accuracy:.2f}%")

        for flip in range(num_flips):
            layer_name, param_index, bit_position, original_value, flipped_value = self.inject_random_bit_flip()
            new_accuracy = self.evaluate(dataloader)

            result = {
                "layer_name": layer_name,
                "param_index": param_index,
                "bit_position": bit_position,
                "original_value": original_value,
                "flipped_value": flipped_value,
                "new_accuracy": new_accuracy
            }
            results.append(result)
            print(f"Flip {flip+1}: Layer {layer_name}, Param {param_index}, Bit {bit_position}, "
                  f"Accuracy: {new_accuracy:.2f}%")

        self.reset_weights()
        return results


In [7]:
from datasets import load_dataset
from torch.utils.data import DataLoader, Dataset
from transformers import DistilBertTokenizer

# Custom Dataset class for tokenized IMDB data
class IMDBDataset(Dataset):
    def __init__(self, dataset_split, tokenizer, max_length=128):
        self.data = dataset_split
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = self.data[idx]["text"]
        label = self.data[idx]["label"]
        encoded = self.tokenizer(text, truncation=True, padding='max_length',
                                 max_length=self.max_length, return_tensors="pt")
        return {
            "input_ids": encoded["input_ids"].squeeze(),
            "attention_mask": encoded["attention_mask"].squeeze(),
            "label": torch.tensor(label, dtype=torch.long)
        }

# Load data and preprocess
def get_dataloader(tokenizer, split="test", batch_size=16):
    dataset = load_dataset("imdb", split=split)
    dataset = dataset.map(lambda x: {"label": 1 if x["label"] == "pos" else 0})  # Binary labels: pos=1, neg=0
    dataset = dataset.shuffle(seed=42)
    imdb_dataset = IMDBDataset(dataset, tokenizer)
    return DataLoader(imdb_dataset, batch_size=batch_size, shuffle=True)

# Instantiate the evaluator
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
dataloader = get_dataloader(tokenizer, split="test", batch_size=16)

evaluator = TransformerEvaluator()

# Run the experiment
results = evaluator.run_experiment(dataloader, num_flips=10)

# Print results
for result in results:
    print(result)


Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


AttributeError: 'str' object has no attribute 'to'